In [4]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

___

In [5]:
# import parasail
# result = parasail.sw_scan_16("asdf", "asdf", 11, 1, parasail.blosum62)
# result = parasail.sw_stats_striped_8("asdf", "asdf", 11, 1, parasail.pam100)

In [6]:
# builtin libraries
import re
import logging
from typing import Union, Optional
from pathlib import Path
from collections import namedtuple, defaultdict
from functools import partial

# secondary libraries
import numpy as np
import parasail
import torch
import torch.nn as nn
from torch.utils.data import DataLoader # load batches to the network
from fast_ctc_decode import beam_search, viterbi_search
from tqdm import tqdm

# feito 
# from basecaller_tester import BasecallerTester as Tester
from feito.models import SimpleNet, Rodan
from feito.dataloaders.dataloader import DatasetONT
# from feito.callbacks import CSVLogger, ModelCheckpoint
# ---- 

# types
_Path = Union[Path,str]

class BasecallerTester:
    
    split_cigar = re.compile(r"(?P<len>\d+)(?P<op>\D+)")
    
    def __init__(self, model, device, test_loader, path_fasta: Optional[_Path] = None, rna: bool = True, use_viterbi = True):
        self.model  = model.to(device) # model with pretrained weigths loaded
        self.device = device
        self.test_loader = test_loader # load signals
        self.batch_size  = test_loader.batch_size
        self.path_fasta  = path_fasta # to save basecalled raw-reads (if not None)
        self.rna = rna 
        self.alphabet    = "NACGU" if rna else "NACGT"
        self.use_viterbi = use_viterbi
        self.search_algo = viterbi_search if use_viterbi  else beam_search

        # set evaluation/inference mode
        self.model.eval()

    def __call__(self,):
        print("Que me dice")
        # TODO: implement tester, considering accuracy

        # 1. generate output of a signal

        # 2. call viterbi or beam search to generate portion of a read
        
        # 3. send to a fasta file

        return ""
    
    def basecall_one_batch(self, X):
        "Return basecalled signals in the chosen alphabet"
        preds  = self.model(X) # preds shape: (len-signal, item, size-alphabet)
        basecalled_signals = (self.signal_to_read(signal=preds[:,item,:], use_viterbi=self.use_viterbi, rna=self.rna) for item in range(self.batch_size))

        return basecalled_signals

    def label_to_alphabet(self, label):
        "Map vector of integers to sequence"
        int2char = {i:c for i,c in zip([1,2,3,4], self.alphabet.replace("N",""))}
        return "".join([int2char[i] for i in label if i > 0])
    
    def accuracy_one_batch(self, batch):
        X, y, output_len, target_len = (x.to(self.device) for x in batch)

        basecalled_signals = self.basecall_one_batch(X)
        ground_truth = np.apply_along_axis(lambda l: self.label_to_alphabet(l), 1, y.detach().numpy()) 
        accuracy_batch = [self.accuracy(ref=gt, seq=bs) for gt,bs in zip(basecalled_signals, ground_truth)]
        
        return accuracy_batch, basecalled_signals
    
    def accuracy_all_dataset(self,):
        basecalled_signals = []
        accuracies = []
        n_batches=len(self.test_loader)
    
        with tqdm(total=n_batches, leave=True, ncols=100, bar_format='{l_bar}{bar}| [{elapsed}{postfix}]') as progress_bar:

            for n_batch, batch in enumerate(self.test_loader):
                
                # Description for progress bar
                progress_bar.set_description(f"Evaluating | Batch: {n_batch+1}/{n_batches}")
                
                # Compute accuracy
                accuracy_batch, basecalled_signals_batch = self.accuracy_one_batch(batch)
                accuracies.extend(accuracy_batch)

                basecalled_signals.extend(basecalled_signals_batch)
                # progress_bar.set_postfix(train_loss='%.4f' % current_avg_loss)
                progress_bar.update(1)

        return accuracies, basecalled_signals

    def signal_to_read(signal, use_viterbi: bool = True, rna: bool = True):
        "Apply viterbi or beam search to a batch"
        alphabet = "NACGU" if rna else "NACGT"
        
        if use_viterbi is True:
            seq, path = viterbi_search(signal, alphabet) 
        else:
            seq, path = beam_search(signal, alphabet, beam_size=5, beam_cut_threshold=0.1)

        return seq

    def accuracy(self, ref, seq, balanced=False, min_coverage=0.0):
        # From https://github.com/nanoporetech/bonito/blob/655feea4bca17feb77957c7f8be5077502292bcf/bonito/util.py#L354
        """
        Calculate the accuracy between `ref` and `seq`
        """
        # alignment = parasail.sw_trace_striped_32(seq, ref, 8, 4, parasail.dnafull)
        alignment = parasail.sw_trace(seq, ref, 8, 4, parasail.dnafull)
        counts = defaultdict(int)

        q_coverage = len(alignment.traceback.query) / len(seq)
        r_coverage = len(alignment.traceback.ref) / len(ref)

        if r_coverage < min_coverage:
            return 0.0

        _, cigar = self.parasail_to_sam(alignment, seq)

        for count, op  in re.findall(self.split_cigar, cigar):
            counts[op] += int(count)

        if balanced:
            accuracy = (counts['='] - counts['I']) / (counts['='] + counts['X'] + counts['D'])
        else:
            accuracy = counts['='] / (counts['='] + counts['I'] + counts['X'] + counts['D'])
        return accuracy * 100


    def parasail_to_sam(self, result, seq):
        # From https://github.com/nanoporetech/bonito/blob/655feea4bca17feb77957c7f8be5077502292bcf/bonito/util.py#L321
        """
        Extract reference start and sam compatible cigar string.

        :param result: parasail alignment result.
        :param seq: query sequence.

        :returns: reference start coordinate, cigar string.
        """
        cigstr = result.cigar.decode.decode()
        first = re.search(self.split_cigar, cigstr)

        first_count, first_op = first.groups()
        prefix = first.group()
        rstart = result.cigar.beg_ref
        cliplen = result.cigar.beg_query

        clip = '' if cliplen == 0 else '{}S'.format(cliplen)
        if first_op == 'I':
            pre = '{}S'.format(int(first_count) + cliplen)
        elif first_op == 'D':
            pre = clip
            rstart = int(first_count)
        else:
            pre = '{}{}'.format(clip, prefix)

        mid = cigstr[len(prefix):]
        end_clip = len(seq) - result.end_query - 1
        suf = '{}S'.format(end_clip) if end_clip > 0 else ''
        new_cigstr = ''.join((pre, mid, suf))
        return rstart, new_cigstr        

In [7]:
Args=namedtuple("Args", ["path_test", "batch_size", "model", "path_checkpoint", "device"])
args = Args(
"../data/subsample_val.hdf5",
16,
"SimpleNet",
"../output/training/checkpoints/SimpleNet-epoch1.pt",
None,
)
    
PATH_TEST=args.path_test
BATCH_SIZE=args.batch_size
MODEL=args.model
DEVICE=args.device
PATH_CHECKPOINT=args.path_checkpoint

if DEVICE is None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else: 
    device = DEVICE
print("Device" , device)

model=eval(f"{MODEL}()")
model.to(device)
if device.type == "cpu":
    model.load_state_dict(torch.load(PATH_CHECKPOINT, map_location=torch.device('cpu')))
else: 
    model.load_state_dict(torch.load(PATH_CHECKPOINT))
model_output_len = model.output_len

# dataset
dataset_test = DatasetONT(recfile=PATH_TEST, output_network_len=model_output_len)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)

/home/javila/micromamba/envs/basecalling-cuda117/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1680572619157/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Device cpu


In [8]:
dataloader_test.batch_size

16

In [9]:
device.type

'cpu'

In [10]:
tester=BasecallerTester(
    model=model, 
    device=device,
    test_loader=dataloader_test,
    path_fasta="output/testing/basecalled_reads.fa"
)

In [11]:
# inference
# tester.accuracy(
seq, ref = ("ACGTACGTACGTACGAGCAT","ACGACTACGACTACACACAC")
result = parasail.sw_trace(seq, ref, 8, 4, parasail.dnafull)

In [12]:
rstart, new_cigstr = tester.parasail_to_sam(result, seq)

In [13]:
rstart, new_cigstr

(0, '4S3=1I2=1I5=4S')

In [ ]:
batch = next(iter(dataloader_test))
model.eval()
X, y, output_len, target_len = (x.to(device) for x in batch)
preds = model(X)

In [ ]:
X.shape, preds.shape, y.shape

In [ ]:
int2char = {i:c for i,c in zip([1,2,3,4],"ACGT")}
b = np.apply_along_axis(lambda l: "".join([int2char[i] for i in l if i > 0]), 1, y.detach().numpy()) 
b.shape, b

In [ ]:
def signal_to_read(signal, use_viterbi: bool = True, rna: bool = True):
    "Apply viterbi or beam search to a batch"
    alphabet = "NACGU" if rna else "NACGT"
    if use_viterbi is True:
        print("using viterbi")
        seq, path = viterbi_search(signal, alphabet) 
    else:
        # beam search
        print("using beam search")
        seq, path = beam_search(signal, alphabet, beam_size=5, beam_cut_threshold=0.1)

    return seq

In [ ]:
batch_signals = preds.detach().numpy()
print(batch_signals.shape)


In [ ]:
batch_signals.transpose((0,2,1)).shape

In [ ]:
np.apply_along_axis(signal_to_read, -1, batch_signals.transpose((0,2,1)))

In [ ]:
[signal_to_read(batch_signals[:,i,:]) for i in range(16)]

In [ ]:
alphabet = "NACGT"
posteriors = np.random.rand(100, len(alphabet)).astype(np.float32)

In [ ]:
posteriors.shape

In [ ]:
viterbi_search(batch_signals[:,0,:], alphabet="NACGU") 